# whole operation for a given symbol:

In [1]:
import os
import numpy as np
import sqlite3
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import yfinance as yf

class BCC:

    def __init__(self):
        self.db_name = 'positions.db'
        self.create_positions_db()
        self.DATA = dict()

    ## Position's database
    def create_positions_db(self, file_path = "tickers-nasdaq.txt"):
        connection, cursor = self.connect_db()

        cursor.execute('''
                        CREATE TABLE IF NOT EXISTS tickers (
                            id INTEGER PRIMARY KEY,
                            ticker TEXT NOT NULL
                        );''')

        tickerSymbols = []
        with open(file_path, 'r') as file:
            for line in file:
                tickerSymbols.append(line.strip())

        for symbol in tickerSymbols:
            cursor.execute('''INSERT INTO tickers (ticker) VALUES (?)''',
                           (str(symbol),)
                          )
            connection.commit()
            
            cursor.execute('''
                            CREATE TABLE IF NOT EXISTS data_{} (
                                id INTEGER PRIMARY KEY,
                                ticker_id TEXT NOT NULL,
                                date DATETIME,
                                close FLOAT,
                                pred FLOAT,
                                induced_high FLOAT,
                                induced_low FLOAT,
                                position INTEGER,
                                entry_price FLOAT,
                                exit_price FLOAT,
                                duration INTEGER,
                                gross_pnl FLOAT,
                                fee FLOAT,
                                net_pnl FLOAT,
                                FOREIGN KEY (ticker_id) REFERENCES tickers (id)
                            );'''.format(symbol))
    
        self.commit_close_db()

    def insert_data_db(self, symbol, data: dict):
        connection, cursor = self.connect_db()

        cursor.execute('SELECT id FROM tickers WHERE ticker = ?', (data[symbol]["ticker"],))
        ticker_id = cursor.fetchone()
        ticker_id = ticker_id[0]

        if data[symbol]["entry_price"] == None and data[symbol]["exit_price"] == None:
            cursor.execute("INSERT OR REPLACE INTO data_{} (ticker_id, date, close, pred, induced_high, induced_low, position) VALUES (?,?,?,?,?,?,?)".format(data[symbol]["ticker"]),
                            (
                                int(ticker_id),
                                str(data[symbol]["date"]),
                                float(data[symbol]["close"]),
                                float(data[symbol]["pred"]),
                                float(data[symbol]["induced_high"]),
                                float(data[symbol]["induced_low"]),
                                float(data[symbol]["position"]),
                                #float(data[symbol]["entry_price"]),
                                #float(data[symbol]["exit_price"]),
                                #int(data[symbol]["duration"]),
                                #float(data[symbol]["gross_pnl"]),
                                #float(data[symbol]["fee"]),
                                #float(data[symbol]["net_pnl"]),
                            )
                          )

        if data[symbol]["entry_price"] != None:
            cursor.execute("INSERT OR REPLACE INTO data_{} (ticker_id, date, close, pred, induced_high, induced_low, position, entry_price) VALUES (?,?,?,?,?,?,?,?)".format(data[symbol]["ticker"]),
                            (
                                int(ticker_id),
                                str(data[symbol]["date"]),
                                float(data[symbol]["close"]),
                                float(data[symbol]["pred"]),
                                float(data[symbol]["induced_high"]),
                                float(data[symbol]["induced_low"]),
                                float(data[symbol]["position"]),
                                float(data[symbol]["entry_price"]),
                                #float(data[symbol]["exit_price"]),
                                #int(data[symbol]["duration"]),
                                #float(data[symbol]["gross_pnl"]),
                                #float(data[symbol]["fee"]),
                                #float(data[symbol]["net_pnl"]),
                            )
                          )

        if data[symbol]["exit_price"] != None:
            cursor.execute("INSERT OR REPLACE INTO data_{} (ticker_id, date, close, pred, induced_high, induced_low, position, exit_price, duration, gross_pnl, fee, net_pnl) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)".format(data[symbol]["ticker"]),
                            (
                                int(ticker_id),
                                str(data[symbol]["date"]),
                                float(data[symbol]["close"]),
                                float(data[symbol]["pred"]),
                                float(data[symbol]["induced_high"]),
                                float(data[symbol]["induced_low"]),
                                float(data[symbol]["position"]),
                                #float(data[symbol]["entry_price"]),
                                float(data[symbol]["exit_price"]),
                                int(data[symbol]["duration"]),
                                float(data[symbol]["gross_pnl"]),
                                float(data[symbol]["fee"]),
                                float(data[symbol]["net_pnl"]),
                            )
                          )

        self.commit_close_db()

    def get_prev_position_db(self, symbol):
        connection, cursor = self.connect_db()

        cursor.execute('SELECT position FROM data_{} ORDER BY date DESC LIMIT 1'.format(symbol))
        position = cursor.fetchone()

        if not position:
            return 0
        else:
            return position[0]

    def get_date_of_last_entry_db(self, symbol, today=pd.Timestamp.today().normalize(), timezone='America/New_York'):
        connection, cursor = self.connect_db()

        cursor.execute('SELECT date FROM data_{} WHERE entry_price IS NOT NULL ORDER BY date DESC LIMIT 1'.format(symbol))
        last_entry_date = cursor.fetchone()

        if not last_entry_date:
            return today - pd.Timedelta(days=1)
        else:
            last_entry_date = pd.Timestamp(f'{last_entry_date[0]}')
            last_entry_date = last_entry_date.tz_localize(timezone)
            
            return last_entry_date

    def get_price_of_last_entry_db(self, symbol):
        connection, cursor = self.connect_db()

        cursor.execute('SELECT entry_price FROM data_{} WHERE entry_price IS NOT NULL ORDER BY date DESC LIMIT 1'.format(symbol))
        last_entry_price = cursor.fetchone()

        if not last_entry_price:
            return 0.
        else:
            return last_entry_price[0]

    def check_position_change_db(self, symbol):
        connection, cursor = self.connect_db()

        cursor.execute('SELECT position FROM data_{} ORDER BY date DESC LIMIT 2'.format(symbol))
        position_new = cursor.fetchone()
        position_new = position_new[0]
        position_old = cursor.fetchone()

        if not position_old:
            position_old = 0
        else:
            position_old = position_old[0]
        
        if position_new != position_old:
            return True
        else:
            return False

    def export_to_excel_db(self, file_path="tickers-nasdaq.txt"):
        connection, cursor = self.connect_db()

        output_directory = 'bcc-bot-output'
        os.makedirs(output_directory, exist_ok=True)
        
        tickerSymbols = []
        with open(file_path, 'r') as file:
            for line in file:
                tickerSymbols.append(line.strip())

        for symbol in tickerSymbols:
            df = pd.read_sql_query(f"SELECT * FROM data_{symbol}", connection)
            df.to_excel(f'{output_directory}/data_{symbol}.xlsx', index=False)
            
        connection.close()
    
    def connect_db(self):
        import sqlite3
        self.connection = sqlite3.connect(self.db_name)
        self.cursor = self.connection.cursor()

        return self.connection, self.cursor

    def commit_close_db(self):
        self.connection.commit()
        self.connection.close()

    ## Creating sequences
    @staticmethod
    def create_dataset(dataset, time_step=1, output_step=1):   
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-output_step):
            a = dataset[i:(i+time_step), 0]
            b = dataset[(i+time_step):(i+time_step)+output_step, 0]
            dataX.append(a)
            dataY.append(b)
    
        return np.array(dataX), np.array(dataY)

    ## Is working day?
    @staticmethod
    def is_working_day(date, country='US'):
        import holidays
        
        are_holidays = holidays.CountryHoliday(country)
        return date.weekday() < 5 and date not in are_holidays

    ## Initialized trained model
    @staticmethod
    def initialize_model(units = 1024,
                         input_period = 46,
                         num_features = 1,
                         output_step = 7,
                         best_epoch_backup = 94,
                         model_weights_path = 'model_weights_2'
                        ):
        # Build LSTM model
        model = Sequential([
            LSTM(units=units, input_shape=(input_period, num_features)),
            Dense(output_step)
        ])
        
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')
    
        # Load the weights of the model at the chosen epoch
        model.load_weights(f'{model_weights_path}/model_weights_epoch_{best_epoch_backup:02d}.h5')
        print('Backup: Weigths for the best epoch has been loaded.')
    
        return model

    ## Process given ticker
    def process_ticker(self,
                       tickerSymbol,
                       start = pd.Timestamp.today().normalize() - pd.Timedelta(days=100),
                       end = pd.Timestamp.today().normalize(),
                       interval = '1d',
                       #file_path = "tickers-nasdaq.txt",
                       trend_period = 14,
                       input_period = 46,
                       period = 60,
                       model = initialize_model(),
                       timezone = 'America/New_York',
                       verbose = False
                      ):
        
        if type(start) != pd.Timestamp:
            start = pd.Timestamp(f'{start} 00:00:00')
            start = start.tz_localize(timezone)
    
        if end is None:
            pass
        else:
            if type(end) != pd.Timestamp:
                end = pd.Timestamp(f'{end} 00:00:00')
                end = end.tz_localize(timezone)
    
        tickerData = yf.Ticker(tickerSymbol)
        df = tickerData.history(start=start, end=end, interval=interval, auto_adjust=False)
        df = df.tail(period)
    
        # Select only the necessary columns
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        df.rename(columns={'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)
        df.rename_axis('date', inplace=True)

        if verbose:
            print(f"Raw data for {tickerSymbol} has been loaded")
    
        from statsmodels.tsa.seasonal import seasonal_decompose
        
        X = df[['close']].to_numpy()
        decomposition = seasonal_decompose(X, model='additive', period=trend_period)
        trend = decomposition.trend
        trend = trend[~np.isnan(trend)]
    
        from sklearn.preprocessing import MinMaxScaler
    
        scaler = MinMaxScaler()
        trend_scaled = scaler.fit_transform(trend.reshape(-1,1))
    
        pred = model.predict(trend_scaled.reshape(trend_scaled.shape[1], input_period, 1), verbose=0)
        pred = scaler.inverse_transform(pred)

        if verbose:
            print(f"Predictions for {tickerSymbol} has been generated")
    
        noise_high = df[['high']].to_numpy()
        noise_low = df[['low']].to_numpy()
        std_dev_high = np.std(noise_high)
        std_dev_low = np.std(noise_low)
        std_dev_noise = np.maximum(std_dev_high, std_dev_low)
    
        induced_high = pred[0] + std_dev_noise
        induced_low = pred[0] - std_dev_noise
    
        return {
            'ticker': tickerSymbol,
            'date': end.strftime('%Y-%m-%d %H:%M:%S'),
            'close': df['close'].iloc[-1],
            'pred': pred[0][0],
            'induced_high': induced_high[0],
            'induced_low': induced_low[0],
            'position': None,
            'entry_price': None,
            'exit_price': None,
            'duration': None,
            'gross_pnl': None,
            'fee': None,
            'net_pnl': None
        }

    def process_all_tickers(self,
                            file_path = "tickers-nasdaq.txt",
                            today = pd.Timestamp.today().normalize(),
                            timezone = 'America/New_York'
                           ):
        if type(today) != pd.Timestamp:
            today = pd.Timestamp(f'{today} 00:00:00')
            today = today.tz_localize(timezone)
            
        tickerSymbols = []
        with open(file_path, 'r') as file:
            for line in file:
                tickerSymbols.append(line.strip())

        from tqdm import tqdm
        for symbol in tqdm(tickerSymbols):
            self.DATA[symbol] = self.process_ticker(symbol, start=today - pd.Timedelta(days=100), end=today)
            self.DATA[symbol]['position'] = (self.process_ticker(symbol, start=today - pd.Timedelta(days=100), end=today - pd.Timedelta(days=1))['induced_low'] < self.DATA[symbol]['induced_low']).astype(int)

            if self.DATA[symbol]['position'] == 1 and self.get_prev_position_db(symbol) == 0:
                self.DATA[symbol]['entry_price'] = self.DATA[symbol]['close']

            if self.DATA[symbol]['position'] == 0 and self.get_prev_position_db(symbol) == 1:
                self.DATA[symbol]['exit_price'] = self.DATA[symbol]['close']
                self.DATA[symbol]['duration'] = (today - self.get_date_of_last_entry_db(symbol, today=today)).days
                self.DATA[symbol]['gross_pnl'] = (self.DATA[symbol]['exit_price'] - self.get_price_of_last_entry_db(symbol)) / self.get_price_of_last_entry_db(symbol)
                self.DATA[symbol]['fee'] = self.DATA[symbol]['gross_pnl'] * 0.002
                self.DATA[symbol]['net_pnl'] = self.DATA[symbol]['gross_pnl'] - self.DATA[symbol]['fee']

            self.insert_data_db(symbol, self.DATA)

        # check position changes
        message = []
        for symbol in tickerSymbols:
            if self.check_position_change_db(symbol):
                if self.DATA[symbol]['position'] == 1:
                    message.append(f"Open position for {symbol} with LONG")
                elif self.DATA[symbol]['position'] == 0:
                    message.append(f"Close position {symbol} with SHORT")
            else:
                if self.DATA[symbol]['position'] == 1:
                    message.append(f"Hold position for {symbol}")
                elif self.DATA[symbol]['position'] == 0:
                    message.append(f"No buy signal for {symbol}")
        if not message:
            message.append("No news")

        #self.send_mail(message, today=today)
            
        return self.DATA

    def send_mail(self, text: str, today=pd.Timestamp.today().normalize()):
        import smtplib
        from email.mime.text import MIMEText
        from email.mime.multipart import MIMEMultipart
        
        # Email credentials
        sender_email = "emailadress"
        sender_password = "password"
        recipient_email = ["emailadress", "emailadress"]
        
        # Create a message
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = ", ".join(recipient_email)
        message["Subject"] = "BCC BOT - Daily Notification Mail"
        
        # Add body to the email
        body = f"TODAY'S REPORT for {today}" + "\n"
        for t in text:
            body += t + "\n"
        message.attach(MIMEText(body, "plain"))
        
        # Connect to the SMTP server
        with smtplib.SMTP("smtp.office365.com", 587) as server:
            server.starttls()  # Start TLS encryption
            server.login(sender_email, sender_password)  # Login to the SMTP server
            text = message.as_string()  # Convert the message to a string
            server.sendmail(sender_email, recipient_email, text)  # Send the email
            print("Email sent successfully!")            

2024-05-21 07:05:45.934842: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 07:05:46.016438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 07:05:46.016513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 07:05:46.019634: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 07:05:46.034117: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 07:05:47.553972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

Backup: Weigths for the best epoch has been loaded.


In [2]:
bot = BCC()

In [3]:
_date = pd.Timestamp("2023-12-04")
_date = _date.tz_localize('America/New_York')

In [4]:
res = []
for delta in range(50):
    if BCC.is_working_day(_date + pd.Timedelta(days=delta)):
        day = bot.process_all_tickers(today=_date + pd.Timedelta(days=delta))
        res.append(day)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [01:01<00:00,  1.60it/s]


In [5]:
bot.export_to_excel_db()